In [ ]:
import pandas as pd
from pathlib import Path
import csv
import plotly.graph_objects as go
import plotly.express as px
from yahoo_fin import stock_info as si

In [ ]:
# Get ticker list
tickers_path = "Resources/tickers_list/tickers_list.csv"
with open(tickers_path) as csvfile:
    data = csv.reader(csvfile)
    next(data)
    tickers_list = [row[0] for row in data]
analysis_code = tickers_list[-1]
tickers_list.pop()

### Display Summary of filtered stocks (Maybe make a new df for easy scanning)

# Display tickers
print("List of the tickers")
print(tickers_list)

In [ ]:
# Display which filter(s) did the application use
analysis_code = 5 - int(analysis_code)
conditions = ["Passing Golden Cross", "20-day SMA >= 50-day SMA", "20-day EMA >= 20-day SMA", "RSI >= 50", "Closing Price >= 20-day EMA"]
print("List of technical analysis applied to filter")
print(conditions[:analysis_code])

In [ ]:
# Display candlestick
for ticker in tickers_list:
    csvpath = f"Resources/{ticker}.csv"
    df = pd.read_csv(csvpath, index_col=0, infer_datetime_format=True, parse_dates=True).dropna()
    df = df.reset_index()

    fig = go.Figure(
        layout_title_text=ticker + ' (July 2020 - September 2022)'
    )
    
    fig.add_trace(
        go.Candlestick(
                x=df['index'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name=ticker
        ))
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 50'],
            mode='lines',
            name='SMA 50'
        )    
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 200'],
            mode='lines',
            name='SMA 200'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['EMA 20'],
            mode='lines',
            name='EMA 20'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 20'],
            mode='lines',
            name='SMA 20',
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['RSI'],
            mode='lines',
            name='RSI',
            yaxis='y2'
        )
    )

    fig.update_layout(
        xaxis_rangeslider_visible=False,
        yaxis_title='Price',
        width=900,
        height=600,
        autosize=True,
        yaxis_domain=[0.2, 1],
        yaxis2={"domain": [0, 0.15]}
    )

    # Display figure
    display(fig)

    # Display summaries like cumulative return, closing price, volume, etc
    # Display fundamentals like P/E ratio
    quote = si.get_quote_table(ticker)
    fund_series = pd.Series(quote)
    fund_df = pd.DataFrame(fund_series)
    fund_df.columns = [ticker]
    display(fund_df)
    